In [1]:
from common import DATA_DIR, Districts
from os.path import join
import pandas as pd
EXCEL_DATA = join(DATA_DIR, "Quartiere","DistrictData_20.07.21.xlsx")
# All units in kW
# All costs in EUR
# All emissions in kgCO2 per kWh

In [2]:
year = 2019
start = f"{year}-01-01"
days = 365  # Full year

# Correct for leap year
if year % 4 == 0:
    days += 1

# Set first the datetime objects with the appropriate dates
dates = pd.date_range(start, periods=days * 24*4+1, freq="15T")
district = Districts.ENAQ.value["name"]

# Electricity Demand
d_el = pd.read_excel(EXCEL_DATA,
                            "2.power demand series",
                            engine='openpyxl')#["DE01"][2:].tolist()

d_el=d_el.drop([0,1]) # first two rows are empyt
d_el=d_el.rename(columns={d_el.columns[0]:"Step"}) 
d_el["Step"]= [int(x) for x in d_el["Step"]]
d_el.set_index("Step", inplace=True)
d_el = d_el[[district]]
d_el=d_el.dropna(axis=0)
d_el= d_el[district]*4 # this is power demand so I want this in kW
d_el

Step
0         50.331968
1         50.331968
2         50.182304
3         49.882978
4         49.433987
            ...    
35035    122.777601
35036    101.912093
35037     88.001754
35038     72.041409
35039     54.031057
Name: ENaQ, Length: 35040, dtype: object

In [3]:
# thermal demand
d_th = pd.read_excel(EXCEL_DATA,
                            "1.heat demand series",
                            engine='openpyxl')
d_th=d_th.drop([0,1]) # first two rows are empyt
d_th=d_th.rename(columns={d_th.columns[0]:"Step"}) 
d_th["Step"]= [int(x) for x in d_th["Step"]]
d_th.set_index("Step", inplace=True)
d_th = d_th[[district]]
d_th=d_th.dropna(axis=0)
d_th= d_th[district]*4
d_th

Step
0        142.075744
1        142.075744
2        140.990365
3        138.819606
4        135.563467
            ...    
35035     50.828506
35036     31.967765
35037     19.393937
35038     11.650688
35039      8.738016
Name: ENaQ, Length: 35040, dtype: object

In [4]:
# PV Production  and Wind Production per kW installed (as data source, no direct Modeling)
vol_prof = pd.read_excel(EXCEL_DATA,
                            "2.volatile series",
                            engine='openpyxl')
vol_prof=vol_prof.drop([0,1]) # first two rows are empyt

vol_prof=vol_prof.rename(columns={vol_prof.columns[0]:"Step"}) 
vol_prof["Step"]= [int(x) for x in vol_prof["Step"]]
vol_prof.set_index("Step", inplace=True)

col_names = list(vol_prof.columns)
for i in range(len(col_names)):
    if i%2==0:
        try:
            vol_prof=vol_prof.rename(columns={col_names[i+1]:f"{col_names[i]}-Wind"}) 
        except:
            pass
district = Districts.ENAQ.value["name"]
solar_prof = vol_prof[district] # This does need correction since are production profiles per kW installed
wind_prof = vol_prof[f"{district}-Wind"] # This does need correction since are production profiles per kW installed

In [5]:
hp_cop = pd.read_excel(EXCEL_DATA,
                            "1.heat pump efficiency series",
                            engine='openpyxl')
hp_cop=hp_cop.drop([0,1]) # first two rows are empyt
hp_cop=hp_cop.rename(columns={hp_cop.columns[0]:"Step"}) 
hp_cop["Step"]= [int(x) for x in hp_cop["Step"]]
hp_cop.set_index("Step", inplace=True)
hp_cop = hp_cop[[district]]
hp_cop=hp_cop.dropna(axis=0)
hp_cop= hp_cop[district]


In [6]:
# All list need to have the same lenght
min_len = min (len(dates),
len(d_el),
len(d_th),
len(wind_prof),
len(solar_prof),
len(hp_cop))

dates = dates[0:min_len]
d_el = d_el[0:min_len]
d_th = d_th[0:min_len]
wind_prof = wind_prof[0:min_len]
solar_prof = solar_prof[0:min_len]
hp_cop = hp_cop[0:min_len]



In [7]:
# get configuration parameters:
# this fetches the data from the worksheets and builds a configuration dictionary to be passed to the oemof model

configuration = {}

# fetch the emission factor as well
configuration["name"] = district
configuration["datetime"]=dates
heat_sources = pd.read_excel(EXCEL_DATA,
                            "0.commodity sources",
                            engine='openpyxl')



In [8]:
gas_grid=heat_sources[heat_sources["fuel"]=="natural gas"]
gas_grid.set_index("District", inplace=True)

configuration["gas"]={}
configuration["gas"]["fuel"] = gas_grid.at[district,"fuel"]
configuration["gas"]["emission"] = gas_grid.at[district,"emission"]

ext_heat=heat_sources[heat_sources["fuel"]=="heat grid"]
if district in ext_heat.index:
    configuration["ext_grid"]={}
    configuration["ext_grid"]["type"] = gas_grid.at[district,"fuel"]
    configuration["ext_grid"]["emission"] = gas_grid.at[district,"emission"]


In [9]:
heat_plants = pd.read_excel(EXCEL_DATA,
                            "1.heat plants",
                            engine='openpyxl')

heat_plants=heat_plants[heat_plants["District"].notna()]

boilers = heat_plants[heat_plants["Technology"]=="boiler"]
hps = heat_plants[heat_plants["Technology"]=="heat pump"]

boilers.set_index("District", inplace=True)
hps.set_index("District", inplace=True)

if boilers.at[district,"capacity"]>0:
    configuration["boiler"]={}
    configuration["boiler"]["p_kw"]=boilers.at[district,"capacity"]*1000
    configuration["boiler"]["eff"]=boilers.at[district,"efficiency"]

if hps.at[district,"capacity"]>0:
    configuration["heat_pump"]={}
    configuration["heat_pump"]["p_kw"]=hps.at[district,"capacity"]*1000
    configuration["heat_pump"]["cop"]= hp_cop




In [10]:
heat_demand = pd.read_excel(EXCEL_DATA,
                            "1.heat demand total",
                            engine='openpyxl')

heat_demand=heat_demand[heat_demand["District"].notna()]
heat_demand.set_index("District", inplace=True)
configuration["heat_demand"]={}
configuration["heat_demand"]["total_kWh"]=heat_demand.at[district,"Total"]

# d_th is in kW and to sum that it needs to be in quarter hours
configuration["heat_demand"]["profile"]= d_th/(d_th.sum()/4) * configuration["heat_demand"]["total_kWh"]


In [11]:
# pass the reneewables  configurations with factors
renewables = pd.read_excel(EXCEL_DATA,
                            "2.volatile power plants",
                            engine='openpyxl')
renewables=renewables[renewables["District"].notna()]
renewables.set_index("District", inplace=True)

if renewables.at[district,"PV"]>0:
    configuration["PV"]={}
    configuration["PV"]["p_kw"]=renewables.at[district,"PV"]
    configuration["PV"]["profile"]=solar_prof*renewables.at[district,"PV"]

if renewables.at[district,"Solar Thermal"]>0:
    configuration["ST"]={} # solar thermal
    configuration["ST"]["a_m2"]=renewables.at[district,"Solar Thermal"]
    # Solar thermal has an average anuual yield of 150 kWh thermal per m2
    # This needs to be normalized 
    total_production = configuration["ST"]["a_m2"]*150  # kwH
    configuration["ST"]["total"] = total_production
    configuration["ST"]["profile"]=solar_prof/(solar_prof.sum()/4)*total_production

if renewables.at[district,"Wind"]>0:
    configuration["Wind"]={}
    configuration["Wind"]["p_kw"]=renewables.at[district,"Wind"]
    configuration["Wind"]["profile"]=wind_prof*renewables.at[district,"Wind"]



In [12]:
el_demand = pd.read_excel(EXCEL_DATA,
                            "2. Total Electricity Demand",
                            engine='openpyxl')

el_demand=el_demand[el_demand["District"].notna()]
el_demand.set_index("District", inplace=True)
configuration["el_demand"]={}
configuration["el_demand"]["total_kWh"]=el_demand.at[district,"Total Electricity"]

# d_th is in kW and to sum that it needs to be in quarter hours
configuration["el_demand"]["profile"]= d_el/(d_el.sum()/4) * configuration["el_demand"]["total_kWh"]


In [13]:
# chp
chps = pd.read_excel(EXCEL_DATA,
                            "3.heat-chp plants",
                            engine='openpyxl')
chps=chps[chps["District"].notna()]
chps.set_index("District", inplace=True)

if chps.at[district,"capacity"]>0:
    configuration["chp"]={}
    configuration["chp"]["p_kw"]=chps.at[district,"capacity"]
    configuration["chp"]["eff_th"]=chps.at[district,"efficiency_heat"]
    configuration["chp"]["eff_el"]=chps.at[district,"efficiency_el"]




In [14]:
# electrolyser
electrolyzers = pd.read_excel(EXCEL_DATA,
                            "4.H2 plant",
                            engine='openpyxl')
electrolyzers=electrolyzers[electrolyzers["District"].notna()]
electrolyzers.set_index("District", inplace=True)

if electrolyzers.at[district,"capacity"]>0:
    configuration["electrolizer"]={}
    configuration["electrolizer"]["p_kw"]=chps.at[district,"capacity"]*1000
    configuration["electrolizer"]["eff_th"]=chps.at[district,"efficiency_heat"]
    configuration["electrolizer"]["eff_h2"]=chps.at[district,"efficiency_el"]



In [15]:
heat_sto = pd.read_excel(EXCEL_DATA,
                            "1.heat storage",
                            engine='openpyxl')
heat_sto=heat_sto[heat_sto["District"].notna()]
heat_sto.set_index("District", inplace=True)


if heat_sto.at[district,"capacity"]>0:	

    configuration["heat_sto"]={}
    configuration["heat_sto"]["capacity"]=heat_sto.at[district,"capacity"]*1000
    configuration["heat_sto"]["charge_cap"]=heat_sto.at[district,"charge capacity"]*1000
    configuration["heat_sto"]["discharge_cap"]=heat_sto.at[district,"discharge capacity"]*1000
    configuration["heat_sto"]["charge_eff"]=heat_sto.at[district,"charge efficiency"]
    configuration["heat_sto"]["disch_eff"]=heat_sto.at[district,"discharge efficiency"]
    configuration["heat_sto"]["loss_rate"]=heat_sto.at[district,"loss rate"]
    configuration["heat_sto"]["volume"]=heat_sto.at[district,"volume"]
    configuration["heat_sto"]["delta_temp"]=heat_sto.at[district,"delta temp"]



In [16]:
batt_sto = pd.read_excel(EXCEL_DATA,
                            "2.power storages",
                            engine='openpyxl')
batt_sto=batt_sto[batt_sto["District"].notna()]
batt_sto.set_index("District", inplace=True)


if batt_sto.at[district,"capacity"]>0:	

    configuration["batt_sto"]={}
    configuration["batt_sto"]["capacity"]=batt_sto.at[district,"capacity"]*1000
    configuration["batt_sto"]["charge_cap"]=batt_sto.at[district,"charge capacity"]*1000
    configuration["batt_sto"]["discharge_cap"]=batt_sto.at[district,"discharge capacity"]*1000
    configuration["batt_sto"]["charge_eff"]=batt_sto.at[district,"charge efficiency"]
    configuration["batt_sto"]["disch_eff"]=batt_sto.at[district,"discharge efficiency"]
    configuration["batt_sto"]["loss_rate"]=batt_sto.at[district,"loss rate"]


In [17]:
configuration

{'name': 'ENaQ',
 'datetime': DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 00:15:00',
                '2019-01-01 00:30:00', '2019-01-01 00:45:00',
                '2019-01-01 01:00:00', '2019-01-01 01:15:00',
                '2019-01-01 01:30:00', '2019-01-01 01:45:00',
                '2019-01-01 02:00:00', '2019-01-01 02:15:00',
                ...
                '2019-12-31 21:30:00', '2019-12-31 21:45:00',
                '2019-12-31 22:00:00', '2019-12-31 22:15:00',
                '2019-12-31 22:30:00', '2019-12-31 22:45:00',
                '2019-12-31 23:00:00', '2019-12-31 23:15:00',
                '2019-12-31 23:30:00', '2019-12-31 23:45:00'],
               dtype='datetime64[ns]', length=35040, freq='15T'),
 'gas': {'fuel': 'natural gas', 'emission': 0.2},
 'boiler': {'p_kw': 1000000, 'eff': 0.98},
 'heat_pump': {'p_kw': 150.0,
  'cop': Step
  0         1.688
  1        1.6855
  2        1.6828
  3         1.678
  4         1.672
            ...  
  35035    1.7884
 